In [2]:
from dotmap import DotMap
import json
import pandas as pd
import numpy as np
import matplotlib.patches as mpatches
import seaborn as sns
import neptune.new as neptune
import os
import datetime

"""
High quality plots for evaluation.

author(s): Arnold Unterauer
"""




###########################



algo_qtran = [2112, 2113, 2114, 2115, 2116]
algo_qmix = [2117, 2118, 2119, 2120, 2121]
algo_qplex = [2122, 2123, 2124, 2125, 2126]
algo_ow_qmix = [2127, 2128, 2129, 2130, 2131]
algo_iql = [2132, 2133, 2134, 2135, 2136]
algo_vdn = [2137, 2138, 2139, 2140, 2141]


hidden_qtran = [2172, 2173, 2174, 2175, 2176]

obvious_qtran = [2177, 2178, 2179, 2180, 2181]
obvious_qmix = [2182, 2183, 2184, 2185, 2186]




algo_qtran_eval = [2554, 2555, 2556, 2557, 2558]
algo_qmix_eval = [2559, 2560, 2561, 2562, 2563]
algo_qplex_eval = [2564, 2565, 2566, 2567, 2568]
algo_vdn_eval = [2569, 2570, 2571, 2572, 2573]
algo_iql_eval = [2574, 2575, 2576, 2577, 2578]
algo_ow_qmix_eval = [2579, 2580, 2581, 2582, 2583]



titel = "Training so_many_baneling QTRAN"

runs = [algo_qmix, algo_vdn, algo_iql, algo_qplex, algo_ow_qmix, algo_qtran]
labels = ["QMIX", "VDN", "IQL", "QPLEX", "OW-QMIX", "QTRAN"]



legend_location = "upper left"

# plot_types = ["ag_0_same_a_p"]
# plot_types = ["return", "win"]
# plot_types = ["return", "win", "accuracy_train_total", "accuracy_test_total"]
# plot_types = ["return", "win", "accuracy_train_total", "accuracy_test_total", "loss_dis", "loss_ant"]
# plot_types = ["return", "win", "ally_health", "enemy_health", "accuracy_train_total", "accuracy_test_total", "accuracy_step_total",  "ag_0_same_a_p"]
plot_types = ["return", "win", "ally_health", "enemy_health"]

# plot_types = ["accuracy_train_total", "accuracy_test_total", "accuracy_step_total", "ally_health", "enemy_health"]

x_ticks = [0 , 80001]
flatui = ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71", "#43a16f"]

In [ ]:
for value_name in plot_types:

    if value_name == "return":
        y_range = [-0.1, 22]
        x_range = [-10, 80000]
        x_tick_steps = 20000
        legend_location = "lower right"
    elif value_name == "accuracy_train_total":
        y_range = [0.45, 1.01]
        x_range = [-10, 10010]
        x_tick_steps = 5000
        legend_location = "upper left"
    elif value_name == "accuracy_test_total":
        y_range = [0.45, 1.01]
        x_range = [-10, 8010]
        x_tick_steps = 2000
        legend_location = "upper left"
    elif value_name == "accuracy_step_total":
        y_range = [0.45, 1.01]
        x_range = [-10, 8010]
        x_tick_steps = 2000
    elif value_name == "win":
        y_range = [-0.01, 1.01]
        x_range = [-10, 80000]
        x_tick_steps = 20000
        legend_location = "upper left"
    elif value_name == "loss_dis":
        y_range = [-0.01, 5.01]
        x_range = [-10, 460]
        x_tick_steps = 100
        legend_location = "upper left"
    elif value_name == "loss_ant":
        y_range = [-0.01, 5.01]
        x_range = [-10, 460]
        x_tick_steps = 100
        legend_location = "upper left"
    elif value_name == "enemy_health":
        y_range = [-0.01, 1.01]
        x_range = [-10, 80000]
        x_tick_steps = 10000
        legend_location = "upper right"
    elif value_name == "ag_0_same_a_p":
        y_range = [-0.01, 1.01]
        x_range = [-10, 80000]
        x_tick_steps = 20000
        legend_location = "upper left"
    elif value_name == "ally_health":
        y_range = [-0.01, 1.01]
        x_range = [-10, 80000]
        x_tick_steps = 10000
        legend_location = "upper right"
    else:
        y_range = [-0.1, 22]
        x_range = [-10, 80000]
        x_tick_steps = 10000

    epoch_return = None
    df = None

    with open('neptune_auth.json', 'r') as f:
        neptune_auth = json.load(f)
        neptune_auth = DotMap(neptune_auth)

    def get_df_from_neptune(run_id):
        neptune_run = neptune.init(project=neptune_auth.project, api_token=neptune_auth.api_token, mode="read-only", run="AA-{}".format(run_id))
        epoch_return = neptune_run[value_name].fetch_values(include_timestamp=False)['value']
        df = pd.DataFrame()
        df["x"] = range(1, len(epoch_return) + 1)
        df[value_name] = epoch_return.rolling(100).mean()
        neptune_run.stop()
        return df

    dataframes = []
    for run in runs:
        dataframe = []
        for nep_id in run:
            dataframe.append(get_df_from_neptune(nep_id))
        dataframe = pd.concat(dataframe, ignore_index=True)
        dataframes.append(dataframe)

    current_palette = sns.color_palette(flatui)

    plt = None
    for i, dataframe in enumerate(dataframes):
        plt = sns.lineplot(x="x", y=value_name, data=dataframe, color=current_palette[i], err_style='band', ci="sd")

    plt.set_xlabel('episode')
    plt.set_ylabel(value_name)
    plt.set_xticks(np.arange(x_ticks[0], x_ticks[1], step=x_tick_steps))

    colors = []
    if len(labels) == 0:
        labels = runs
    for i, label in enumerate(labels):
        colors.append(mpatches.Patch(color=current_palette[i], label='{}'.format(label)))

    leg = plt.legend(handles=colors, labelspacing=0.1, loc=legend_location)
    leg.set_title(titel)

    plt.set(xlim=(x_range[0], x_range[1]))
    plt.set(ylim=(y_range[0], y_range[1]))
    fig = plt.get_figure()

    timestamp = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

    plot_dir = os.path.join(os.getcwd(), 'plots')
    if not os.path.exists(plot_dir):
        os.makedirs(plot_dir)
    fig.savefig(os.path.join(plot_dir, 'plot - {}.png'.format(timestamp)), dpi=300, bbox_inches="tight")
    fig.clf()